# FESDModel

FESD - Fault estimation for skeleton detection - is a suite that aims at finding faults in joints of skeletons, which are detected by human pose estimatiors.

FESDData is the sister project to this notebook, which aims at recording depth and rgb data, as well as populating the data with human poses from variing human pose estimators.

Furthermore, FESTData augments all data based on joint confidence.

FFESDModel aims to develop and evaluate a model based on the faulty and augmented joint data as well as RGBD data.

## Libraries

We need a range of libraries which are imported here. We also define some constants.

In [119]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tqdm

import cv2
import pyrealsense2 as rs

sns.set_style()

In [120]:
RECORDING_DIR = 'H:/Recordings/'
EXERCISES_FILE_NAME = 'Exercises.json'

## Data Loading

Firstly we need to import all the recordings into the notebook.


### Load Metadata

It is important to load the metadata, such as the session parameters, the exercises and the recording paths.

In [121]:
recording_jsons = []
for file in os.listdir(RECORDING_DIR):
  if (file.endswith('.json') and 
      not file.endswith('Skeleton.json') and 
      not file.endswith(EXERCISES_FILE_NAME)):
    with open(file=os.path.join(RECORDING_DIR, file), mode='r') as file:
      data = json.load(file)
      recording_jsons.append(data)

len(recording_jsons)

20

Then we load the Exercises from the exercise file.

In [122]:
exercises_json = []

with open(file=os.path.join(RECORDING_DIR, EXERCISES_FILE_NAME), mode='r') as file:
  exercises_json = json.load(file)['Exercises']

len(exercises_json)

13

Next we extract all recordings from the sessions.

In [123]:
recording_paths_rs = []
recording_paths_orbbec = []

for recording in recording_jsons:
  for camera in recording['Cameras']:
    if camera['Type'] == 'Realsense':
      recording_paths_rs.append(camera['FileName'])

len(recording_paths_rs)

20

### Load Frames from Realsense Camera

Next we load the frames from the realsense recordings into arrays of CV matrixes.


In [133]:
def load_recording_rs(recording_path_rs, num_frames: int):
  pipeline = rs.pipeline()
  config = rs.config()

  rs.config.enable_device_from_file(config, os.path.join(RECORDING_DIR, recording_path_rs))
  
  # Configure the pipeline to stream the depth stream
  # Change this parameters according to the recorded bag file resolution
  config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
  config.enable_stream(rs.stream.color, 1280, 720, rs.format.rgb8, 30)

  # Start streaming from file
  pipeline.start(config)
  device = pipeline.get_active_profile().get_device()
  
  device.as_playback().set_real_time(False)
    
  depth_scale = device.first_depth_sensor().get_depth_scale()
  
  frames_to_return = []
  
  align = rs.align(rs.stream.depth)

  # Streaming loop
  for i in tqdm.tqdm(range(num_frames)):

    # Get frameset of depth
    frames = pipeline.wait_for_frames()
    frames = align.process(frames)
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()
    
    if not depth_frame or not color_frame:
      print("Warning: Frame missing!")
      continue

    color_frame = np.asanyarray(color_frame.get_data())
    color_frame = cv2.cvtColor(color_frame, cv2.COLOR_BGR2RGB) / 255

    depth_frame = np.asanyarray(depth_frame.get_data())
    depth_frame.shape = (480, 640, 1)

    
    
    frame = np.append(color_frame, depth_frame * depth_scale, axis=2)
    
    frames_to_return.append(frame)
  pipeline.stop()

  return frames_to_return

In [134]:
frames = []

for (i, recording_path_rs) in enumerate(recording_paths_rs):
  print(f'Loading recording {i+1} of {len(recording_paths_rs)} for recording {recording_jsons[i]["Name"]}')
  frames.append(load_recording_rs(recording_path_rs, recording_jsons[i]['Frames']))

print(f"Depth Range: {frames[0][0][:,:,3:].min()} - {np.ceil(frames[0][0][:,:,3:].max())}")
print(f"Color Range: {frames[0][0][:,:,:3].min()} - {frames[0][0][:,:,:3].max()}")

Loading recording 1 of 20 for recording Session 2023-02-08T22:21:33


100%|██████████| 602/602 [00:10<00:00, 57.05it/s]


(480, 640, 4)


In [136]:
cv2.namedWindow('Samples', cv2.WINDOW_AUTOSIZE)

sample_size = 300

for vid in frames:
  counter = 0
  for frame in vid:
    counter += 1
    if counter > sample_size:
      cv2.imshow('Samples', images)
      break
    
    color_image = frame[:,:,:3]
    depth_image = frame[:,:,3:]
    depth_image.shape = depth_image.shape[:2]
    
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=255/8), cv2.COLORMAP_INFERNO) / 255
    
    depth_colormap_dim = depth_colormap.shape
    color_colormap_dim = color_image.shape
    
    images = np.hstack((color_image, depth_colormap))

    cv2.imshow('Samples', images)
    cv2.waitKey(1)
        
cv2.destroyAllWindows()

[[[0.01568627 0.         0.        ]
  [0.01568627 0.         0.        ]
  [0.01568627 0.         0.        ]
  ...
  [0.43137255 0.09803922 0.43529412]
  [0.43137255 0.09411765 0.42745098]
  [0.43137255 0.09411765 0.42745098]]

 [[0.01568627 0.         0.        ]
  [0.01568627 0.         0.        ]
  [0.01568627 0.         0.        ]
  ...
  [0.43137255 0.09803922 0.43529412]
  [0.43137255 0.09411765 0.42745098]
  [0.43137255 0.09411765 0.42745098]]

 [[0.01568627 0.         0.        ]
  [0.01568627 0.         0.        ]
  [0.01568627 0.         0.        ]
  ...
  [0.43137255 0.09803922 0.43529412]
  [0.43137255 0.09411765 0.42745098]
  [0.43137255 0.09411765 0.42745098]]

 ...

 [[0.01568627 0.         0.        ]
  [0.01568627 0.         0.        ]
  [0.01568627 0.         0.        ]
  ...
  [0.01568627 0.         0.        ]
  [0.01568627 0.         0.        ]
  [0.01568627 0.         0.        ]]

 [[0.01568627 0.         0.        ]
  [0.01568627 0.         0.        ]


### Load Skeleton data

Next we load the human pose estimation data. The human pose estimation data is stored in a json.

Should I load the skeleton data as an image with a heatmap based on the confidence rating?